In [6]:
import os
import numpy as np
import scipy.stats
import pandas as pd
import sklearn.neural_network
# import plotting libraries
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('seaborn') # pretty matplotlib plots

import seaborn as sns
sns.set('notebook', font_scale=1.25, style='whitegrid')


In [7]:
def convolve2(img, rows, cols, kernel):
    x_pad = (kernel.shape[1]-1)//2
    y_pad = (kernel.shape[0]-1)//2

    padded = np.zeros((rows + 2*y_pad, cols + 2*x_pad))
    padded[y_pad:y_pad+rows, x_pad:x_pad+cols] = img.copy()

    out = np.zeros((rows, cols))
    
    x_size = kernel.shape[1]
    y_size = kernel.shape[0]

    for r in range(rows):
        for c in range(cols):
            out[r,c] = np.sum(kernel * padded[r:r+y_size, c:c+x_size])

    return out


In [40]:



DATA_DIR = 'data_sneaker_vs_sandal' #Make sure you have downloaded data and your directory is correct

df_x_test = pd.read_csv(os.path.join(DATA_DIR, 'x_test.csv'))
df_x_train = pd.read_csv(os.path.join(DATA_DIR, 'x_train.csv'))
df_y_train = pd.read_csv(os.path.join(DATA_DIR, 'y_train.csv'))


x_test = df_x_test.to_numpy()
x_train = df_x_train.to_numpy()
y_train = df_y_train.to_numpy()

#INSERT TRANSFORM BELOW
trans_train = []

for row in x_train:
    k = np.ones((3,3))
    curr = np.where(row > 0, 1, 0)
    curr = convolve2(curr.reshape((28,28)), 28, 28, k)
    

    trans_train.append(curr.flatten())

trans_train = np.array(trans_train)
x_train = np.hstack([x_train, trans_train])


#INSERT TRANSFORM ABOVE

ind = np.arange(x_train.shape[0])
np.random.shuffle(ind)
split_size = x_train.shape[0]//2
x_valid = x_train[ind[: split_size]]
y_valid = y_train[ind[: split_size]]
x_train = x_train[ind[split_size:]]
y_train = y_train[ind[split_size:]]

print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

mlp = sklearn.neural_network.MLPClassifier(
    hidden_layer_sizes=[32],
    solver='adam',
    max_iter=1000)

# We can create a *distribution* object using scipy.stats

dist = scipy.stats.randint(4, 64)

my_parameter_distributions_by_name = dict(
    hidden_layer_sizes=scipy.stats.randint(20, 120),
    alpha=scipy.stats.uniform(0.0, 1.0),
    random_state=[  # try two possible seeds to initialize parameters
        69, 420
        ],
    )

n_trials_rand_search = 40

xall_N2 = np.vstack([x_train, x_valid])
yall_N = np.vstack([y_train, y_valid])

print(xall_N2.shape)
print(yall_N.shape)


valid_indicators_N = np.hstack([
    -1 * np.ones(y_train.size), # -1 means never include this example in any test split
    0  * np.ones(y_valid.size), #  0 means include in the first test split (we count starting at 0 in python)
    ])
    
# Create splitter object using Predefined Split

my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_N)

my_rand_searcher = sklearn.model_selection.RandomizedSearchCV(
    mlp,
    my_parameter_distributions_by_name,
    scoring="accuracy",
    cv=my_splitter,
    n_iter=n_trials_rand_search,
    random_state=101, # same seed means same results everytime we repeat this code
    )

my_rand_searcher.fit(xall_N2, yall_N.flatten())

param_keys = ['param_hidden_layer_sizes', 'param_alpha', 'param_random_state']

# Rearrange row order so it is easy to skim
rsearch_results_df = pd.DataFrame(my_rand_searcher.cv_results_).copy()
rsearch_results_df.sort_values(param_keys, inplace=True)


print("Dataframe has shape: %s" % (str(rsearch_results_df.shape)))

print("Dataframe has columns:")
for c in rsearch_results_df.columns:
    print("-- %s" % c)

print(rsearch_results_df[param_keys + ['split0_test_score', 'rank_test_score']])



(6000, 1568)
(6000, 1)
(6000, 1568)
(6000, 1)
(12000, 1568)
(12000, 1)
Dataframe has shape: (40, 12)
Dataframe has columns:
-- mean_fit_time
-- std_fit_time
-- mean_score_time
-- std_score_time
-- param_alpha
-- param_hidden_layer_sizes
-- param_random_state
-- params
-- split0_test_score
-- mean_test_score
-- std_test_score
-- rank_test_score
   param_hidden_layer_sizes param_alpha param_random_state  split0_test_score  \
16                       20    0.712598                 69           0.978833   
25                       22    0.575205                 69           0.977167   
4                        25    0.189939                 69           0.979333   
22                       28    0.164303                 69           0.982000   
6                        28    0.785602                420           0.977167   
17                       37    0.839463                 69           0.979000   
8                        39    0.603548                 69           0.953667   
36    

In [41]:
bestg_mlp = mlp.set_params(**my_rand_searcher.best_params_)
print(bestg_mlp)

bestg_mlp.fit(x_train, y_train.flatten())

print(bestg_mlp.score(x_valid, y_valid.flatten()))

MLPClassifier(alpha=0.20916149886092483, hidden_layer_sizes=66, max_iter=1000,
              random_state=420)
0.9825


In [ ]:
#transformation with binarized data as addition feature with 3x3
# MLPClassifier(alpha=0.5542275911247871, hidden_layer_sizes=42, max_iter=1000,
#               random_state=202)
# 0.9811666666666666

In [ ]:
#transformation with 3x3 and 1x5
# MLPClassifier(alpha=0.14623958613215304, hidden_layer_sizes=56, max_iter=1000,
#               random_state=420)
# 0.9828333333333333

trans_train = []
trans_train2 = []
for row in x_train:
    k_2 = np.ones((3,3))
    k = np.ones((1,5))
    curr = np.where(row > 0, 1, 0)
    curr1 = convolve2(curr.reshape((28,28)), 28, 28, k)
    curr2 = convolve2(curr.reshape((28,28)), 28, 28, k_2)
    trans_train.append(curr1.flatten())
    trans_train2.append(curr2.flatten())

trans_train = np.array(trans_train)
trans_train2 = np.array(trans_train2)
x_train = np.hstack([x_train, trans_train, trans_train2])


In [ ]:
#transformation, weighted matrix
# MLPClassifier(alpha=0.14623958613215304, hidden_layer_sizes=56, max_iter=1000,
#               random_state=420)
# 0.9825

trans_train = []

for row in x_train:
    k = np.ones((3,3))
    k[0,0] = 2
    k[0,2] = 2
    k[2,0] = 2
    k[2,2] = 2
    k[1,0] = 1.5
    k[1,2] = 1.5
    k[0,1] = 1.5
    k[2,1] = 1.5
    curr = np.where(row > 0, 1, 0)
    curr = convolve2(curr.reshape((28,28)), 28, 28, k)
    

    trans_train.append(curr.flatten())

trans_train = np.array(trans_train)
x_train = np.hstack([x_train, trans_train])
